### ИСТОЧНИКИ

https://www.kaggle.com/tunguz/russian-glove/downloads/russian-glove.zip/1



In [105]:
import collections
import nltk
import os
import numpy as np

In [258]:
def stream_sentences(src = "data/poems.txt"):
    file_path_src = src
    allHaiku = []
    with open(file_path_src, encoding="utf-8") as f:
        lines = f.readlines()
        by3lines = []
        for line in lines:
            if line.strip() == "":
                allHaiku.append(by3lines)
                by3lines = []
            else:
                by3lines.append(line.lower())
    return allHaiku

In [6]:
def to_oneline(data):
    return ["".join(row) for row in data]

In [264]:
def get_sentences(file):
    return to_oneline(stream_sentences(file))

In [460]:
sents = get_sentences("data/poems_fix.txt")

Download glove file from Kaggle to data directory

https://www.kaggle.com/tunguz/russian-glove

TODO: request file from shareable URL

In [90]:
def load_glove_vectors(glove_file, embed_size=300):
    words = [
        "PAD",
        "UNK"]
    vects = [
        np.zeros((embed_size)), # PAD
        np.random.uniform(-1, 1, embed_size)] # UNK

    fglove = open(glove_file, "rb")
    for line in fglove:
        cols = line.strip().split()
        word = cols[0].decode('utf-8')
        vect = np.array([float(v) for v in cols[1:]])
        words.append(word)
        vects.append(vect)

    vocab = {w: i for i, w in enumerate(words)}
    return words, vocab, np.array(vects)

In [91]:
words, vocab, embeddings = load_glove_vectors("data/multilingual_embeddings.ru")

In [130]:
import gensim
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_input_file="data/multilingual_embeddings.ru", word2vec_output_file="data/gensim_glove_vectors.txt")


(63068, 300)

In [131]:
from gensim.models.keyedvectors import KeyedVectors
glove_model = KeyedVectors.load_word2vec_format("data/gensim_glove_vectors.txt", binary=False)

In [133]:
glove_model.most_similar("и")

[('оборачиваетесь', 0.7298823595046997),
 ('а', 0.7228611707687378),
 ('черно', 0.654677152633667),
 ('обеспеченные', 0.6393757462501526),
 ('поднимаются', 0.6386812925338745),
 ('мерцающее', 0.6307729482650757),
 ('разбираем', 0.6296200752258301),
 ('маршировать', 0.6277832984924316),
 ('поступая', 0.6192404627799988),
 ('повторяли', 0.6133629679679871)]

In [146]:
glove_model.similar_by_vector(embeddings[vocab["и"]])[1][0]

'оборачиваетесь'

In [80]:
embeddings.shape

(63070, 300)

In [81]:
vocab["и"]

13027

In [82]:
words[13027]

'и'

In [92]:
def safe_word2id(word, vocab):
    return vocab["UNK"] if vocab.get(word) == None else vocab[word]

In [474]:
from keras.preprocessing import sequence

SEQUENCE_LEN = 20

except_words = collections.Counter()
normal_words = {}
tokens_vectors = [[w for w in nltk.word_tokenize(s)] for s in sents]
# verse_index_vectors = [[safe_word2id(w, vocab) for w in s] for s in tokens_vectors]
windex_vectors = []
good_sentences = []
good_verses = []
for s in tokens_vectors:
    sent_vect = []
    unk_count = 0
    for w in s:
        i = safe_word2id(w, vocab)
        sent_vect.append(i)
        if (i == vocab["UNK"]):
            unk_count += 1
            except_words[w] += 1
        else:
            normal_words[w] = 1

    windex_vectors.append(sent_vect)
    if unk_count < 2 and s != []:
        good_sentences.append(sent_vect)
        good_verses.append(s)

# [[safe_word2id(w, vocab) for w in s] for s in tokens_vectors]
verse_index_vectors = sequence.pad_sequences(windex_vectors, SEQUENCE_LEN)

In [475]:
len(good_sentences)

1789

In [476]:
np.savetxt("data/good.txt", sequence.pad_sequences(good_sentences, SEQUENCE_LEN), delimiter="\t")
# r = np.genfromtxt("data/index.txt", delimiter="\t")

In [462]:
len(normal_words.keys())

14092

In [463]:
len(except_words.keys())

16676

In [262]:
def get_common_exceptions(sents, n = 40):
    except_words = collections.Counter()
    normal_words = {}
    tokens_vectors = [[w for w in nltk.word_tokenize(s)] for s in sents]
    # verse_index_vectors = [[safe_word2id(w, vocab) for w in s] for s in tokens_vectors]
    windex_vectors = []
    for s in tokens_vectors:
        sent_vect = []
        for w in s:
            i = safe_word2id(w, vocab)
            sent_vect.append(i)
            if (i == vocab["UNK"]):
                except_words[w] += 1
            else:
                normal_words[w] = 1

        windex_vectors.append(sent_vect)

    return except_words.most_common(n)

In [211]:
def tokens2index(tokens_vectors, seq_len = SEQUENCE_LEN):
    windex_vectors = [[safe_word2id(w, vocab) for w in s] for s in tokens_vectors]
    sequence.pad_sequences(windex_vectors, seq_len)

def index2vectors(index_vectors):
    return embeddings[index_vectors]

def vectors2tokens(vectors):
    for sent in vectors:
        print(" ".join([glove_model.similar_by_vector(word_vect)[0][0] for word_vect in sent]))

In [228]:
def replacex2y(x, y):
    file_path = 'data/poems_fix.txt'
    s = ""
    with open(file_path, 'r') as myfile:
      s = myfile.read()
    s = s.replace(x, y)
    with open(file_path, 'w') as myfile:
      myfile.write(s)

In [412]:
def a2b(b, a):
    for n in a:
        replacex2y(" " + n + " ", " " + b + " ")

In [231]:
replacex2y("\n", " \n ")

In [232]:
replacex2y(" што ", " что ")

In [235]:
replacex2y(" говна ", " дерьма ")

In [236]:
replacex2y(" говно ", " дерьмо ")

In [242]:
replacex2y(" хуй ", " хер ")

In [238]:
replacex2y(" исус ", " иисус ")

In [362]:
replacex2y(" исуса ", " иисуса ")

In [245]:
replacex2y(" глядит ", " смотрит ")

In [417]:
replacex2y(" тихонько ", " тихо ")
replacex2y(" тихонечко ", " тихо ")

In [252]:
replacex2y(" чаю ", " чая ")

In [271]:
replacex2y(" жопа ", " опа ")

In [317]:
replacex2y(" жопы ", " мат ")

In [318]:
replacex2y(" жопе ", " мат ")

In [319]:
replacex2y(" сука ", " мат ")

In [320]:
replacex2y(" жопу ", " мат ")

In [390]:
replacex2y(" жопой ", " мат ")

In [359]:
replacex2y(" минет ", " мат ")

In [278]:
replacex2y(" молчит ", " молчат ")

In [281]:
replacex2y(" пиздец ", " конец ")

In [425]:
replacex2y(" глядят ", " смотрят ")
replacex2y(" гляжу ", " смотрю ")
replacex2y(" глядишь ", " глядишь ")

In [288]:
replacex2y(" ево ", " его ")

In [293]:
replacex2y(" чтото ", " что то ")

In [322]:
replacex2y(" гдето ", " где то ")

In [326]:
replacex2y(" ктото ", " кто то ")

In [355]:
replacex2y(" изза ", " из за ")

In [428]:
replacex2y(" както ", " как то ")
replacex2y(" чтонибудь ", " что нибудь ")

In [364]:
replacex2y(" наутро ", " на утро ")

In [337]:
replacex2y(" иль ", " или ")

In [324]:
replacex2y(" щас ", " сейчас ")

In [454]:
replacex2y(" ещо ", " еще ")
replacex2y(" умрем ", " умрём ")

In [328]:
replacex2y(" вобще ", " вообще ")

In [330]:
replacex2y(" покуда ", " пока ")

In [332]:
replacex2y(" чорных ", " чёрных ")

In [342]:
replacex2y(" чорный ", " чёрный ")

In [343]:
replacex2y(" чорные ", " чёрные ")

In [386]:
replacex2y(" чорное ", " чёрное ")

In [336]:
replacex2y(" страшней ", " страшнее ")

In [340]:
replacex2y(" рукою ", " рукой ")

In [427]:
replacex2y(" весною ", " весной ")
replacex2y(" порою ", " порой ")
replacex2y(" зовёт ", " зовет ")
replacex2y(" толпою ", " толпой ")
replacex2y(" своею ", " своей ")

In [433]:
a2b("нечаянно", ["невзначай"])

In [436]:
names = [
 'олег',
 'оксана',
 'аркадий',
 'глеб',
 'олега',
 'николая',
 'геннадий',
 'евгений',
 'семён',
 'петра',
 'зухра',
 'оксану',
 'илья',
 'глеба',
 'олегу',
 'оксане',
 'иван',
 'семен',
 'ольга',
 'анатолий',
 'зульфия',
 'оксаны',
 'пётр',
 'зоя',
 'руфь',
 'вениамин',
 'антон',
 'путин',
 'петру',
 'шаинский',
 'игорь',
 'андрей',
 'семёна',
 'пушкин',
 'гагарин',
 'павел',
 'зухры',
 'илье',
 'оксаной',
 'ильич',
 'константин',
 'иннокентий',
 'николаем',
 'зинаида',
 'зульфии',
 'зухре',
 'ольги',
 'николаю',
 'олегом',
 'зои',
 'петрович',
 'аделаида',
 'марина',
 'афанасий',
 'глебу',
 'зульфию',
 'лариса',
 'василий',
 'петр',
 'муму',
 'татьяна',
 'зое',
 'ольгу',
 'ильи',
 'серёжа',
 'маша',
 'юра',
 'друзь',
 'михаил',
 'герасим',
 'игоря',
 'ленин',
 'аркадию',
 'илью',
 'андрея',
 'ильича',
 'саша',
 'эммануил',
 'зухру',
 'иуда',
 'фёдор',
 'боярский',
 'олеге',
 'петыр',
 'алексей',
 'ольгой',
 'кобзон',
 'коля',
 'оксан',
 'ивана',
]

for n in names:
    replacex2y(" " + n + " ", " имя ")

In [434]:
names = [
 'борщ',
 'пельмени',
 'водку',
 'пирожки',
 'пирожок',
 'пельменей',
 'омлет',
 'кастрюлю',
 'борща',
 'солью',
 'колбасы',
 'арбузы',
 'пельменя',
 'пельмень',
 'котлеты',
]

for n in names:
    replacex2y(" " + n + " ", " еда ")

In [413]:
a2b("умер", [
    "помер",
    "сдох",
])

In [456]:
a2b("мат", [
    "срать",
    "мудак",
    "блядь",
    "блять",
    "попу",
    "говном",
])

In [451]:
a2b("коробка", ["гроб"])
a2b("коробки", ["гроба", "гробы"])
a2b("могилы", ["могил"])

In [457]:
list(map(lambda x: x[0], get_common_exceptions(get_sentences("data/poems_fix.txt"), 150)))

['седой',
 'седая',
 'рублей',
 'жених',
 'косой',
 'семнадцать',
 'ночами',
 'подъезда',
 'крикнул',
 'лужи',
 'москву',
 'горько',
 'приснился',
 'сантехник',
 'снится',
 'пятьсот',
 'трамвае',
 'ежа',
 'усталый',
 'гладит',
 'вибратор',
 'зонт',
 'воет',
 'подъезде',
 'трамвай',
 'тапки',
 'молчу',
 'курю',
 'украдкой',
 'попробуй',
 'запел',
 'плюнул',
 'шорох',
 'трусах',
 'москва',
 'горлу',
 'девчонка',
 'палач',
 'шагает',
 'крым',
 'срывает',
 'целовал',
 'мордой',
 'трогают',
 'кончилась',
 'енота',
 'лежим',
 'бомж',
 'раскинув',
 'убей',
 'уставший',
 'трамвая',
 'уснул',
 'воскрес',
 'журавлей',
 'старенький',
 'гуляет',
 'колготки',
 'бредёт',
 'лежишь',
 'промолвил',
 'четырнадцать',
 'поцеловал',
 'ползи',
 'дорешал',
 'стучится',
 'утру',
 'трахаться',
 'плаще',
 'ёж',
 'портфель',
 'бухло',
 'пляшут',
 'крематорий',
 'обнявшись',
 'трусов',
 'буратино',
 'тёплый',
 'цыганка',
 'дедмороза',
 'весенний',
 'гладить',
 'ждёшь',
 'глядишь',
 'буркнул',
 'село',
 'речке',
 

In [472]:
vocab["ну"]

50816

In [ ]:
from keras.preprocessing import sequence

SEQUENCE_LEN = 20

except_words = collections.Counter()
normal_words = {}
tokens_vectors = [[w for w in nltk.word_tokenize(s)] for s in sents]
# verse_index_vectors = [[safe_word2id(w, vocab) for w in s] for s in tokens_vectors]
windex_vectors = []
good_sentences = []
good_verses = []
for s in tokens_vectors:
    sent_vect = []
    unk_count = 0
    for w in s:
        i = safe_word2id(w, vocab)
        sent_vect.append(i)
        if (i == vocab["UNK"]):
            unk_count += 1
            except_words[w] += 1
        else:
            normal_words[w] = 1

    windex_vectors.append(sent_vect)
    if unk_count < 2 and s != []:
        good_sentences.append(sent_vect)
        good_verses.append(s)

# [[safe_word2id(w, vocab) for w in s] for s in tokens_vectors]
verse_index_vectors = sequence.pad_sequences(windex_vectors, SEQUENCE_LEN)

In [ ]:
len(good_sentences)

In [ ]:
np.savetxt("data/good.txt", sequence.pad_sequences(good_sentences, SEQUENCE_LEN), delimiter="\t")
# r = np.genfromtxt("data/index.txt", delimiter="\t")